# `Scraper` de `Datos Editorial UPTC`

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
import re

## `Scraper` de `URLs` 

In [ ]:
#url de libros Uptc
url = 'https://editorial.uptc.edu.co/catalogo.html?product_list_limit=60'
bucle = 'https://editorial.uptc.edu.co/catalogo.html?p=2&product_list_limit=60'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html')


scrap = soup.find('div',class_="sa-product-container-v3 grid")
productos_url = scrap.find_all('li',class_="sa-product-item-v3 sa-product_grid_size")
url_libros = [url.find('a')["href"] for url in productos_url]

urls = pd.DataFrame(url_libros,columns=['url'])

for i in range(2,6):
    url = f'https://editorial.uptc.edu.co/catalogo.html?p={i}&product_list_limit=60'
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html')
    productos_url = soup.find('div',class_="sa-product-container-v3 grid").find_all('li',class_="sa-product-item-v3 sa-product_grid_size")
    url_libros = [url.find('a')["href"] for url in productos_url]
    urls = pd.concat([urls,pd.DataFrame(url_libros,columns=['url'])])


In [ ]:
#urls.to_json('urls.json',index=False)
urls.shape

In [2]:
url_libros = pd.read_csv('urls.csv')

In [74]:
url_libros.shape

(298, 1)

In [64]:
def extraer_palabras_clave(texto):
    # Patrón para encontrar palabras clave y sus valores
    patron = r'([^:]+):\s*([^\n]+)'
    # Encontrar todas las coincidencias
    coincidencias = re.findall(patron, texto)
    
    # Crear diccionario con las palabras clave
    palabras_clave = {}
    for clave, valor in coincidencias:
        # Limpiar espacios en blanco
        clave = clave.strip()
        valor = valor.strip()
        palabras_clave[clave] = valor
    
    return palabras_clave

def extraer_info_colaborador(bloque):
    # Nombre invertido
    nombre_invertido = bloque.split('\n')[0].strip()
    
    # ORCID
    orcid_match = re.search(r'(https://orcid\.org/[^\s]+)', bloque)
    orcid = orcid_match.group(1) if orcid_match else None
    
    # Afiliación
    afiliacion_match = re.search(r'Afiliación\s*\n([^\n]+)', bloque)
    afiliacion = afiliacion_match.group(1).strip() if afiliacion_match else None
    
    # Biografía
    bio_match = re.search(r'Biografía:\s*([\s\S]+?)(?:\n\S|$)', bloque)
    if bio_match:
        biografia = bio_match.group(1).strip()
    else:
        # Si no encuentra el patrón, toma todo después de "Biografía:"
        bio_idx = bloque.find("Biografía:")
        biografia = bloque[bio_idx+10:].strip() if bio_idx != -1 else None
    
    # Email (opcional)
    email_match = re.search(r'[\w\.-]+@[\w\.-]+', bloque)
    email = email_match.group(0) if email_match else None

    return {
        "nombre_invertido": nombre_invertido,
        "orcid": orcid,
        "afiliacion": afiliacion,
        "biografia": biografia,
        "email": email
    }

 

In [ ]:
temp_datos = []
for i in range(298):
    titulos = []
    autores = []
    palabras_clave = []
    precios = []
    descripcion = []


    res = requests.get(url_libros['url'][i])
    soup = BeautifulSoup(res.text,'html')
    time.sleep(5)
    scrap=soup.find('div',class_="product-info-main")
    try:
        titulos.append(scrap.find('div',class_="col-xs-12").find('h2').text)
    except AttributeError:
        continue
    ## busquemos lo autores
    temp = [i for i in scrap.find_all('div',class_="col-xs-12")]
    temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_author' in a['href'] ]
    autores.append(temp1)
    # busca las palabras clave
    temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_keywords' in a['href'] ]
    palabras_clave.append(temp1)
    # DOi
    try:
        temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_keywords' not in a['href'] and  'products_author' not in a['href']]
        if temp1 != []:
            doi = temp1
        else :
            doi = None 
    except Exception as e:
        print(f'No se encontro doi: {e}')
        doi = None

    # precios
    try:
        temp = [a for a in scrap.find_all('div',class_='price-box price-final_price')]
        if temp != "":  # Si hay elementos de precio
            temp1 = [a.text.strip() for a in temp]
            precios.append(temp1)
        else:  # Si no hay elementos de precio
            precios.append(['Descargable gratis'])
    except Exception as e:
        print(f'No se encontro precio: {e}')
        precios.append(['Descargable gratis'])
    # descripcion
    descripcion.append(scrap.find('div',class_='description-wrapper').text)
    # eBook
    try:
        scrap = soup.find('div',class_="product-info-metadata")
        metadata_ebook = extraer_palabras_clave(scrap.text)
    except Exception as e:
        print(f'No se encontro metadata_ebook: {e}')
        metadata_ebook = None
    #impreso
    try:
        scrap = soup.find_all('div',class_="product-info-metadata")[1]
        metadata_imp = extraer_palabras_clave(scrap.text)
    except Exception as e:
        print(f'No se encontro metadata_imp: {e}')
        metadata_imp = None
    #clasificacion de tematica
    dato = soup.find('div','item content').text.replace('\n','')
    clasificacion_tematica = dato
    #tabla de contenido
    a=soup.find('div',class_='product data items').text.find('Tabla de contenido')
    b=soup.find('div',class_='product data items').text.find('Colaboradores')
    dato = soup.find('div',class_='product data items').text
    if a != -1:
        tabla_contenido=dato[a:b]
    else:
        tabla_contenido= None
    ## informacion de autores
    a=soup.find('div',class_='product data items').text.find('Colaboradores')
    dato = soup.find('div',class_='product data items').text
    colaboradores_raw = dato[a:].split('Nombre invertido')[1:]
    colaboradores = []
    for bloque in colaboradores_raw:
        info = extraer_info_colaborador(bloque)
        colaboradores.append(info)

    datos = {
            "title": titulos,
            "colaboradores":autores,
            "precio":precios,
            "palabras clave":palabras_clave,
            "doi":doi,
            "descripcion":descripcion,
            "metadata_ebook":metadata_ebook,
            "metadata_imp":metadata_imp,
            "clasificación_temática":clasificacion_tematica,
            'tabla_contenido':tabla_contenido,  
            'inf_autores' :colaboradores 
            }
    temp_datos.append(datos)
    

In [101]:
import os
# 1. Leer los datos existentes (si el archivo existe)
if os.path.exists('datos.json'):
    with open('datos.json', mode='r', encoding='utf-8') as f:
        datos_existentes = json.load(f)
else:
    datos_existentes = []

# 2. Agregar los nuevos datos (supongamos que tus nuevos datos están en temp_datos)
datos_existentes.extend(temp_tatos1)

# 3. Guardar la lista completa
with open('datos.json', mode='w', encoding='utf-8') as f:
    json.dump(datos_existentes, f, ensure_ascii=False, indent=4)


In [91]:
url_libros['url'][i]

'https://editorial.uptc.edu.co/gpd-el-ano-viejo-9790900532909.html'

In [102]:
len(datos_existentes)

287

In [98]:
temp_tatos1.extend(temp_datos)

## Limpieza de Datos

In [31]:
import pandas as pd
import re
import json

In [21]:
datos = pd.read_json('datos.json')

In [23]:
def procesar_precio(precio_str):
    patron = r'COP \$(\d+\.?\d*)'
    numeros = re.findall(patron, precio_str)
    if numeros:
        # Convertir el número a entero eliminando los puntos
        return numeros
    return 'Descargable'

In [26]:
# Aplicar la función a la columna de precios
datos['precio'] = datos['precio'].apply(lambda x: procesar_precio(str(x)))

In [32]:
# Convertir el DataFrame a una lista de diccionarios
datos_lista = datos.to_dict('records')

# Guardar en JSON con indentación
with open('datos_pre.json', 'w', encoding='utf-8') as f:
    json.dump(datos_lista, f, ensure_ascii=False, indent=4)

# Verificar que se guardó correctamente
print(f"Se guardaron {len(datos_lista)} registros en el archivo JSON")

Se guardaron 287 registros en el archivo JSON


## Save in Mongodb 

In [1]:
!pip install pymongo

  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.5/911.5 kB 2.9 MB/s eta 0:00:00a 0:00:01
Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)


In [34]:
import pymongo
import json

In [35]:
with open('datos_pre.json',mode='r') as f:
    datos =json.load(f)

In [36]:
len(datos)

287

In [37]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient['editorial']
mycol = mydb['libros']

x = mycol.insert_many(datos)
print(x.inserted_ids)

[ObjectId('683860ecec62f7a31bdf4366'), ObjectId('683860ecec62f7a31bdf4367'), ObjectId('683860ecec62f7a31bdf4368'), ObjectId('683860ecec62f7a31bdf4369'), ObjectId('683860ecec62f7a31bdf436a'), ObjectId('683860ecec62f7a31bdf436b'), ObjectId('683860ecec62f7a31bdf436c'), ObjectId('683860ecec62f7a31bdf436d'), ObjectId('683860ecec62f7a31bdf436e'), ObjectId('683860ecec62f7a31bdf436f'), ObjectId('683860ecec62f7a31bdf4370'), ObjectId('683860ecec62f7a31bdf4371'), ObjectId('683860ecec62f7a31bdf4372'), ObjectId('683860ecec62f7a31bdf4373'), ObjectId('683860ecec62f7a31bdf4374'), ObjectId('683860ecec62f7a31bdf4375'), ObjectId('683860ecec62f7a31bdf4376'), ObjectId('683860ecec62f7a31bdf4377'), ObjectId('683860ecec62f7a31bdf4378'), ObjectId('683860ecec62f7a31bdf4379'), ObjectId('683860ecec62f7a31bdf437a'), ObjectId('683860ecec62f7a31bdf437b'), ObjectId('683860ecec62f7a31bdf437c'), ObjectId('683860ecec62f7a31bdf437d'), ObjectId('683860ecec62f7a31bdf437e'), ObjectId('683860ecec62f7a31bdf437f'), ObjectId('6